In [1]:
r"""
Graphormer pretrained model alignment"""

'\nGraphormer pretrained model alignment'

In [2]:
# select device
import os 
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import os
import json
import logging
import torch
import mindspore as ms
import numpy as np

import transformers.models.graphormer.modeling_graphormer as ptm
import transformers.models.graphormer.configuration_graphormer as ptc

import mindnlp.transformers.models.graphormer.modeling_graphormer as msm
import mindnlp.transformers.models.graphormer.configuration_graphormer as msc

/home/hubo/Software/miniconda3/envs/mindspore/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-23 13:28:35,078 - modelscope - INFO - PyTorch version 1.12.0+cu116 Found.
2023-11-23 13:28:35,079 - modelscope - INFO - Loading ast index from /home/hubo/.cache/modelscope/ast_indexer
2023-11-23 13:28:35,106 - modelscope - INFO - Loading done! Current index file version is 1.9.4, with md5 47d42285eb6171153afc50fa8021c277 and a total number of 945 components indexed


In [5]:
# Convert pytorch_model.bin to ms_graphormer_model.ckpt
def torch_model_to_mindspore(pth_file, **kwargs):
    """Convert pytorch model .bin to mindspore model .ckpt"""
    prefix = kwargs.get('prefix', '')

    try:
        import torch
    except Exception as exc:
        raise ImportError("'import torch' failed, please install torch by "
                          "`pip install torch` or instructions from 'https://pytorch.org'") \
        from exc

    from mindspore.train.serialization import save_checkpoint

    logging.info('Starting checkpoint conversion.')
    ms_ckpt = []
    state_dict = torch.load(pth_file, map_location=torch.device('cpu'))

    for k, v in state_dict.items():
        if 'shared.weight' in k:
            k = k.replace('shared.weight', 'decoder.embed_tokens.embedding_table')
        if 'layer_norm' in k:
            k = k.replace('layer_norm.weight', 'layer_norm.gamma')
            k = k.replace('layer_norm.bias', 'layer_norm.beta')
        if (('encoder.weight' in k) or 
            ('graph_token_virtual_distance.weight' in k) 
            or ('graph_token.weight' in k)):
            k = k.replace('weight', 'embedding_table')
        if prefix:
            k = prefix + "." + k
        print(k)
        ms_ckpt.append({'name': k, 'data': ms.Tensor(v.numpy())})

    ms_ckpt_path = pth_file.replace('pytorch_model.bin','mindspore.ckpt')
    if not os.path.exists(ms_ckpt_path):
        try:
            save_checkpoint(ms_ckpt, ms_ckpt_path)
        except Exception as exc:
            raise RuntimeError(f'Save checkpoint to {ms_ckpt_path} failed, please checkout the path.') \
            from exc

    return ms_ckpt_path


In [6]:
model_dir = '../pretrained_models/graphormer/'
torch_model_to_mindspore(os.path.join(model_dir, 'pytorch_model.bin'))

encoder.graph_encoder.graph_node_feature.atom_encoder.embedding_table
encoder.graph_encoder.graph_node_feature.in_degree_encoder.embedding_table
encoder.graph_encoder.graph_node_feature.out_degree_encoder.embedding_table
encoder.graph_encoder.graph_node_feature.graph_token.embedding_table
encoder.graph_encoder.graph_attn_bias.edge_encoder.embedding_table
encoder.graph_encoder.graph_attn_bias.edge_dis_encoder.embedding_table
encoder.graph_encoder.graph_attn_bias.spatial_pos_encoder.embedding_table
encoder.graph_encoder.graph_attn_bias.graph_token_virtual_distance.embedding_table
encoder.graph_encoder.emb_layer_norm.gamma
encoder.graph_encoder.emb_layer_norm.beta
encoder.graph_encoder.layers.0.self_attn.k_proj.weight
encoder.graph_encoder.layers.0.self_attn.k_proj.bias
encoder.graph_encoder.layers.0.self_attn.v_proj.weight
encoder.graph_encoder.layers.0.self_attn.v_proj.bias
encoder.graph_encoder.layers.0.self_attn.q_proj.weight
encoder.graph_encoder.layers.0.self_attn.q_proj.bias
encode

'../pretrained_models/graphormer/mindspore.ckpt'

In [7]:
# init config
config_path = os.path.join(model_dir, 'config.json')
with open(config_path, encoding='utf-8') as config:
    config = json.load(config)

ms_config = msc.GraphormerConfig(**config)
pt_config = ptc.GraphormerConfig(**config)

In [8]:
# init model
ms_model = msm.GraphormerForGraphClassification(ms_config)
pt_model = ptm.GraphormerForGraphClassification(pt_config)

In [9]:
# load parameters
# load parameters
pt_dict = torch.load(os.path.join(model_dir, 'pytorch_model.bin'))
pt_model.load_state_dict(pt_dict, False) 

ms_dict = ms.load_checkpoint(os.path.join(model_dir, 'mindspore.ckpt'))
param_not_load = ms.load_param_into_net(ms_model, ms_dict)
print(f"Param_not_load:{param_not_load}")

Param_not_load:([], [])


In [10]:
# set eval mode
ms_model.set_train(False)
pt_model.eval()

GraphormerForGraphClassification(
  (encoder): GraphormerModel(
    (graph_encoder): GraphormerGraphEncoder(
      (dropout_module): Dropout(p=0.0, inplace=False)
      (graph_node_feature): GraphormerGraphNodeFeature(
        (atom_encoder): Embedding(4609, 768, padding_idx=0)
        (in_degree_encoder): Embedding(512, 768, padding_idx=0)
        (out_degree_encoder): Embedding(512, 768, padding_idx=0)
        (graph_token): Embedding(1, 768)
      )
      (graph_attn_bias): GraphormerGraphAttnBias(
        (edge_encoder): Embedding(1537, 32, padding_idx=0)
        (edge_dis_encoder): Embedding(131072, 1)
        (spatial_pos_encoder): Embedding(512, 32, padding_idx=0)
        (graph_token_virtual_distance): Embedding(1, 32)
      )
      (emb_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0): GraphormerGraphEncoderLayer(
          (dropout_module): Dropout(p=0.0, inplace=False)
          (activation_dropout_module): Dropout(p=

In [11]:
def ids_tensor_np(shape, vocab_size):
    return np.random.randint(0, vocab_size-1, shape)

In [12]:
def np_inputs_to_ms(inputs):
    inputs_ms = dict()
    for key, value in inputs.items():
        if key == 'attn_bias':
            inputs_ms[key] = ms.tensor(value, dtype=ms.float32)
        else:
            inputs_ms[key] = ms.tensor(value, dtype=ms.int64)
    return inputs_ms

def np_inputs_to_pt(inputs):
    inputs_pt = dict()
    for key, value in inputs.items():
        if key == 'attn_bias':
            inputs_pt[key] = torch.tensor(value, dtype=torch.float32)
        else:
            inputs_pt[key] = torch.tensor(value, dtype=torch.int64)
    return inputs_pt
        

In [13]:
# prepare data
batch_size=10
graph_size=20

inputs = dict(
    input_nodes = ids_tensor_np([batch_size, graph_size, 1], ms_config.num_atoms),
    input_edges = ids_tensor_np([batch_size, graph_size, graph_size, ms_config.multi_hop_max_dist, 1], ms_config.num_edges),
    attn_bias = ids_tensor_np([batch_size, graph_size + 1, graph_size + 1], ms_config.num_atoms),
    in_degree = ids_tensor_np([batch_size, graph_size], ms_config.num_in_degree),
    out_degree = ids_tensor_np([batch_size, graph_size], ms_config.num_out_degree),
    spatial_pos = ids_tensor_np([batch_size, graph_size, graph_size], ms_config.num_spatial),
    attn_edge_type = ids_tensor_np([batch_size, graph_size, graph_size, 1], ms_config.num_edges)
)


inputs_ms = np_inputs_to_ms(inputs)
inputs_pt = np_inputs_to_pt(inputs)

In [14]:
# output
ms_out = ms_model(**inputs_ms)
pt_out = pt_model(**inputs_pt)

In [15]:
ms_out.keys()

odict_keys(['logits', 'hidden_states'])

In [16]:
# Check that the outputs have same shape
assert ms_out['logits'].shape == pt_out['logits'].shape
assert len(ms_out['hidden_states']) == len(pt_out['hidden_states'])

for k in range(len(ms_out['hidden_states'])):
    assert ms_out['hidden_states'][k].shape == pt_out['hidden_states'][k].shape

In [17]:
def judge(o1, o2, loss = 1e-3, prefix = '-'):
    prefix += '-'
    if (isinstance(o1, tuple)):
        for i in range(len(o1)):
            judge(o1[i], o2[i], loss=loss, prefix=prefix)
    elif (isinstance(o1, ms.Tensor)):
        np1 = o1.asnumpy()
        np2 = o2.detach().numpy()
        print(f"{prefix}{np.allclose(np1, np2, loss, loss)}")
        print(np.abs((np1 - np2)).max())
    else:
        print(f"{type(o1)}-{type(o2)}:{o1==o2}")



In [18]:
# Check that the outputs difference are within tolerable range
loss = 5e-3
judge(ms_out['logits'], pt_out['logits'], loss)

for k in range(len(ms_out['hidden_states'])):
    judge(ms_out['hidden_states'][k], pt_out['hidden_states'][k], loss)


--True
0.007780075
--True
0.020700455
--False
0.042342246
--False
0.035519123
--False
0.05296135
--False
0.039479148
--False
0.035338
--False
0.04726553
--False
0.05483198
--False
0.049901724
--False
0.06000042
--False
0.052295685
--False
0.032122612
--False
0.09411812
